<center>

| Nazwa przedmiotu                          | Dataset                                |
| ----------------------------------------- | -------------------------------------- |
| Projekt zespołowy - sztuczna inteligencja | laptop_prices_dataset                  |
|                                           |                                        |
| Piotr                                     | Goraj                                  |
| Numer albumu                              | 55529                                  |
|                                           |                                        |
| Bartosz                                   | Kiałka                                 |
| Numer albumu                              | 55528                                  |
|                                           |                                        |
| Data oddania sprawozdania                 | 2024.11.16                             |
| Kierunek                                  | Informatyka, II stopnia P, Stacjonarne |

</center>


# 1. Instalacja i import potrzebnych bibliotek

In [ ]:
# !pip install pandas
# !pip install scikit-learn
# !pip install numpy

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas import DataFrame

# 2. Odczyt danych

In [ ]:
import pandas as pd

# załadowanie pliku CSV
FILE_PATH = './laptop_prices_dataset.csv'
data = pd.read_csv(FILE_PATH)

In [ ]:
# data info
data.info()

In [ ]:
# pierwsze trzy rekordy
data.head(3)

In [ ]:
# podsumowanie statystyczne
print(data.describe())

In [ ]:
# czy występują wartości null
display(data.isnull().sum())

In [ ]:
# Wybrane kolumny do modeli
selected_columns = [ 'Company','TypeName', 'Price_euros', 'Inches','Ram','OS','Touchscreen','CPU_freq','CPU_model','GPU_company','GPU_model']

# Wczytanie pliku CSV z wybranymi kolumnami
print("Wybrane kolumny CSV:")
df_selected_columns_csv = pd.read_csv(FILE_PATH, usecols=selected_columns)

# pierwsze 10 rekordów
display(df_selected_columns_csv.head(10))

In [ ]:
# Wydzielenie do osobnych plików zbiorów treningowego, walidacyjnego, oraz testowego.

# Podział na zbiór treningowy (70%) oraz zbiór tymczasowy (30% - do podziału na walidacyjny i testowy)
train_data, temp_data = train_test_split(df_selected_columns_csv, test_size=0.3, random_state=42)

# Podział zbioru tymczasowego na walidacyjny (15%) i testowy (15%)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# długość każdego zbioru
prime_len =  len(df_selected_columns_csv)
train_len = len(train_data)
val_len =  len(val_data)
test_len = len(test_data)

print("Długość zbioru:")
print("\tpierwotnego:", prime_len, "(100%)")
print("\ttreningowego:", train_len, "({:.2f}%)".format(train_len * 100 / prime_len))
print("\twalidacyjnego:", val_len, "({:.2f}%)".format(val_len * 100 / prime_len))
print("\ttestowego:", test_len, "({:.2f}%)".format(test_len * 100 / prime_len))

# pierwsze 5 rekordów każdego zbioru
display(train_data.head(5))
display(val_data.head(5))
display(test_data.head(5))

# 3. Przygotowanie danych do modelu

In [ ]:
def replace_with_top_n(data: DataFrame, column: str, n: int, other_label: str ="Other"):
    """
    Zamienia wartości w kolumnie na n najczęściej występujących,
    a pozostałe wartości na 'Other'.
    
    Params:
    - column: Nazwa kolumny do przetworzenia.
    - n: Liczba najczęściej występujących wartości, które mają zostać zachowane.
    - other_label: Etykieta dla pozostałych wartości (domyślnie "Other").

    Returns:
    - DataFrame z przekształconą kolumną.
    """
    top_n = data[column].value_counts().nlargest(n).index
    data[column] = data[column].apply(lambda x: x if x in top_n else other_label)

def replace_with_log_transformation(data: DataFrame, column: str):
    """
    Zamienia wartości w kolumnie na wartości zlogarytmowane.

    Params:
    - column: Nazwa kolumny do przetworzenia.
    """
    data[column] = np.log1p(data[column])

def remove_model_number(data: DataFrame, column: str):
    """
    Usuwa końcowe cyfry (model) z nazwy, pozostawiając tylko część tekstową.
    
    Params:
    - text (str): Tekst do przetworzenia (np. "Core i5 7200U").
    
    Returns:
    - str: Przetworzony tekst bez końcowych cyfr (np. "Core i5").
    """
    data[column] = data[column].apply(lambda text: re.sub(r'\s+\d+\w*$', '', text))

def encode_with_dummies(data: DataFrame, columns: list):
    """
    Przekształca wybrane kolumny na zmienne zero-jedynkowe (one-hot encoding).
    
    Params:
    - data: DataFrame do przekształcenia.
    - columns: Lista kolumn kategorycznych do zakodowania.
    
    Returns:
    - DataFrame z zakodowanymi kolumnami.
    """
    data_encoded = pd.get_dummies(data, columns=columns, drop_first=True)  
    # drop_first zapobiega pułapce zmiennych fikcyjnych 
    # Usuwa pierwszą kolumnę dla każdej kategorii, 
    # aby uniknąć pułapki zmiennych fikcyjnych (tzw. dummy variable trap). 
    # Pomaga to w modelach regresji uniknąć współliniowości.
    return data_encoded


#### 3.1 Dane treningowe

In [ ]:
# top 5 Company
replace_with_top_n(train_data, 'Company', 5)
replace_with_top_n(val_data, 'Company', 5)
replace_with_top_n(test_data, 'Company', 5)

# top 3 TypeName
replace_with_top_n(train_data, 'TypeName', 3)
replace_with_top_n(val_data, 'TypeName', 3)
replace_with_top_n(test_data, 'TypeName', 3)

# top 4 OS
replace_with_top_n(train_data, 'OS', 4)
replace_with_top_n(val_data, 'OS', 4)
replace_with_top_n(test_data, 'OS', 4)

# Top 5 CPU_model
remove_model_number(train_data, 'CPU_model')
remove_model_number(val_data, 'CPU_model')
remove_model_number(test_data, 'CPU_model')

replace_with_top_n(train_data, 'CPU_model', 5)
replace_with_top_n(val_data, 'CPU_model', 5)
replace_with_top_n(test_data, 'CPU_model', 5)

# Top 5 GPU_company
replace_with_top_n(train_data, 'GPU_company', 5)
replace_with_top_n(val_data, 'GPU_company', 5)
replace_with_top_n(test_data, 'GPU_company', 5)

# Top 5 GPU_model
remove_model_number(train_data, 'GPU_model')
remove_model_number(val_data, 'GPU_model')
remove_model_number(test_data, 'GPU_model')

replace_with_top_n(train_data, 'GPU_model', 5)
replace_with_top_n(val_data, 'GPU_model', 5)
replace_with_top_n(test_data, 'GPU_model', 5)

"""
"""

# Transformacja logarytmiczna Ram
replace_with_log_transformation(train_data, 'Ram')
replace_with_log_transformation(val_data, 'Ram')
replace_with_log_transformation(test_data, 'Ram')

# Transformacja logarytmiczna Price_euros
replace_with_log_transformation(train_data, 'Price_euros')
replace_with_log_transformation(val_data, 'Price_euros')
replace_with_log_transformation(test_data, 'Price_euros')

# Transformacja logarytmiczna Inches
replace_with_log_transformation(train_data, 'Inches')
replace_with_log_transformation(val_data, 'Inches')
replace_with_log_transformation(test_data, 'Inches')

# Transformacja logarytmiczna CPU_freq
replace_with_log_transformation(train_data, 'CPU_freq')
replace_with_log_transformation(val_data, 'CPU_freq')
replace_with_log_transformation(test_data, 'CPU_freq')

"""
"""

# One-hot encoding
columns_to_encode = ['OS', 'GPU_company']

train_data_dummies = encode_with_dummies(train_data, columns_to_encode)
val_data_dummies = encode_with_dummies(val_data, columns_to_encode)
test_data_dummies = encode_with_dummies(test_data, columns_to_encode)


# 4. Enkodowanie wartości kategorycznych

In [28]:
def encode_categorical_columns(data: pd.DataFrame):
    """
    Koduje kolumny kategoryczne na numeryczne za pomocą kodowania etykiet (Label Encoding).
    
    Params:
    - data: DataFrame, w którym kolumny kategoryczne mają zostać zakodowane.
    
    Returns:
    - DataFrame z zakodowanymi kolumnami oraz słownik zawierający LabelEncodery dla każdej kolumny.
    """
    data_encoded = data.copy()
    label_encoders = {}
    
    for col in data_encoded.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        data_encoded[col] = le.fit_transform(data_encoded[col])
        label_encoders[col] = le 
    
    return data_encoded, label_encoders

In [29]:
train_data_encoded = train_data_dummies.copy()
val_data_encoded = val_data_dummies.copy()
test_data_encoded = test_data_dummies.copy()

train_data_encoded, train_label_encoders = encode_categorical_columns(train_data_dummies)
val_data_encoded, val_label_encoders = encode_categorical_columns(val_data_dummies)
test_data_encoded, test_label_encoders = encode_categorical_columns(test_data_dummies)

# Wyświetlenie pierwszych 20 rekordów ze zbioru treningowego po kodowaniu
display(train_data_encoded.head(20))


,Company,TypeName,Inches,Ram,Price_euros,Touchscreen,CPU_freq,CPU_model,GPU_model,OS_No OS,OS_Other,OS_Windows 10,GPU_company_ARM,GPU_company_Intel,GPU_company_Nvidia
314,0,2,1.262345,0.741276,1.890156,1,0.554998,0,2,False,False,True,False,True,False
1032,4,0,1.362744,1.162283,2.089894,0,0.848012,3,1,False,False,True,False,False,True
319,4,1,1.262345,0.959135,1.972164,0,0.670652,4,2,False,False,True,False,True,False
1039,2,1,1.310506,1.162283,2.069290,0,0.812157,2,2,False,False,True,False,True,False
966,1,1,1.337472,0.959135,2.043373,0,0.812157,2,2,False,False,True,False,True,False
5,4,1,1.337472,0.959135,1.945047,0,0.869742,4,3,False,False,True,False,False,False
100,2,1,1.337472,1.162283,2.025414,0,0.836526,3,3,False,False,True,False,False,False
713,0,1,1.310506,0.959135,1.902174,0,0.554998,0,2,False,False,True,False,True,False
570,2,2,1.362744,1.162283,2.143519,0,0.848012,2,3,False,False,True,False,False,True
548,2,1,1.337472,0.959135,1.943973,0,0.741276,1,2,False,False,True,False,True,False


# 5. Zapis przygotowanych danych do pliku

In [30]:
# columns = train_data_encoded.columns 
# val_data_encoded = val_data_encoded.reindex(columns=columns, fill_value=0)
# test_data_encoded = test_data_encoded.reindex(columns=columns, fill_value=0)

train_data_encoded.to_csv('train_data.csv', index=False)
val_data_encoded.to_csv('val_data.csv', index=False)
test_data_encoded.to_csv('test_data.csv', index=False)